In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# reading csv files
data =  pd.read_csv(r'D:\HomeWork\Machine Learning\dataset\data_train.csv', sep=",",header = None)
test =  pd.read_csv(r'D:\HomeWork\Machine Learning\dataset\data_test.csv', sep=",",header = None)

goalTest = test.pop(16)
dfTest = test.copy()

decision_trees = []
decision_trees_features = []
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [2]:
def train_using_entropy(X_train, y_train):
    # Decision tree with entropy
    clf_entropy = tree.DecisionTreeClassifier(criterion = "entropy",max_depth = 3)
  
    # Performing training
    clf_entropy.fit(X_train, y_train)
    return clf_entropy

In [3]:
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: \n",
        confusion_matrix(y_test, y_pred))
      
    print ("\n Accuracy : \n",
    accuracy_score(y_test,y_pred)*100)
    
    print ("\n Precision : \n",
    precision_score(y_test,y_pred, average='macro')*100)

In [4]:
# Build each tree of the forest
num_built = 0
while num_built < 15:
    try:
                
        trainData = data.copy()

        goal = trainData.pop(16)
        df = trainData.sample(n=3,axis='columns')
        
        decision_trees_features.append(list(df.columns))
        clf_entropy = train_using_entropy(df, goal)
                
        # Save the classifier
        decision_trees.append(clf_entropy)
        num_built += 1
    except Exception as e:
        continue

In [5]:
# Make predictions with every tree in the forest
X = dfTest
y = []
for i in range(len(decision_trees)):
    tree = decision_trees[i]
    features = decision_trees_features[i]
    dn = list()
    for j in decision_trees_features[i]:
        dn.append(dfTest[j])
    dn = np.array(dn)
    dn = dn.T   
    y.append(tree.predict(dn))
        
        # Reshape so we can find the most common value
y = np.swapaxes(a=y, axis1=0, axis2=1)
        
        # Use majority voting for the final prediction
predictions = []
for preds in y:
    counter = Counter(preds)
    predictions.append(counter.most_common(1)[0][0])

In [6]:
cal_accuracy(goalTest, predictions)

Confusion Matrix: 
 [[291   1   7   0  25   0  34   0   5   0]
 [  1 132 171  31   1   0   1  27   0   0]
 [  0   1 358   2   0   0   0   3   0   0]
 [  1   2   0 332   0   0   0   1   0   0]
 [  8   1   2   6 304   0  40   2   1   0]
 [  0   0   0 161   0 164   5   2   3   0]
 [  1   0  13   9   6   0 306   1   0   0]
 [ 17  17  36  31   1   1   1 260   0   0]
 [ 79   0  78   2   4  21   4  75  73   0]
 [ 18  46   0 259   5   2   5   1   0   0]]

 Accuracy : 
 63.464837049742705

 Precision : 
 64.09273567844407
